Required CSVs: salaries.csv, position.csv

Required jsons: professor_ratings.json

Cal Poly Professor ratings scraped from https://polyratings.dev/search/name. Data collected + example:

| LastName | FirstName | Department | Rating | NumRatings |
| :------- | :-------- | :--------- | :----: | :--------: |
| Doe      | John      | CSC        | 3.5    | 24         |

In [172]:
!pip install itables
import pandas as pd
import requests
import itables
import json
from bs4 import BeautifulSoup
import re

In [173]:
# df_ratings = pd.read_json('professor_ratings.json')
# itables.show(df_ratings)

with open('professor_ratings.json') as f:
    data = json.load(f)

df_ratings = pd.json_normalize(data['data'])
columns = ["lastName", "firstName", "department", "numEvals", "overallRating", "materialClear", "studentDifficulties", "courses"]
df_ratings = df_ratings[columns]

for index, row in df_ratings[(df_ratings['firstName'].str.split().str.len()) > 1].iterrows():
    name = row['firstName']
    firstname, middlename = name.split()
    df_ratings.loc[index, 'firstName'] = firstname

itables.show(df_ratings[columns])

In [174]:
df_ratings

,lastName,firstName,department,numEvals,overallRating,materialClear,studentDifficulties,courses
0,Mahjoor,Parisa,CHEM,17,0.64,0.82,0.45,"[CHEM 110, CHEM 127, CHEM 316]"
1,Clark,Chris,CSC,12,3.70,3.62,3.70,"[CPE 103, CSC 103, CPE 123, CRP 430, CPE 481, ..."
2,LaGuire,Tiev,FSN,13,2.57,2.57,2.71,"[FSN 210, FSN 250, FSN 251]"
3,Lathrop,Amanda,FSN,8,2.22,2.67,1.89,"[FSN 125, FSN 275]"
4,Rucci,Nancy,SPAN,12,2.50,2.50,2.64,"[SPAN 101, SPAN 102, SPAN 111, SPAN 201, SPAN ..."
...,...,...,...,...,...,...,...,...
1857,Slayter,Erik,BUS,71,3.68,3.69,3.70,"[AGB 214, BUS 214, BUS 215, BUS 221, AGB 260, ..."
1858,Neal,Emily,BIO,36,1.94,2.03,1.79,"[BIO 161, BIO 221, MCRO 221]"
1859,Glavan,Matthew,MATH,10,2.64,2.27,2.27,"[MATH 118, MATH 221]"
1860,Jansen,Rebecca,CE,2,1.26,1.01,0.75,[CE 204]


In [175]:
import pandas as pd

df_payroll = pd.read_csv('salaries.csv')

# Extracting first and last name from payroll dataframe to match polyrating name format
for index, row in df_payroll.iterrows():
    name = row['name']
    lastname, firstname_middle = name.split(", ")
    firstname_parts = firstname_middle.split()
    firstname = firstname_parts[0]
    df_payroll.loc[index, 'firstName'] = firstname
    df_payroll.loc[index, 'lastName'] = lastname

df_payroll['avgPay'] = df_payroll['avgPay'].round(2)

In [176]:
# Exctracting out all the periods to ensure data consistency
df_payroll['name'] = df_payroll['name'].str.replace('.', '')
df_payroll

,name,avgPay,firstName,lastName
0,"Aagaard, Sofie J",91650.96,Sofie,Aagaard
1,"Abba, Michele M",57789.41,Michele,Abba
2,"Abelli-Amen, Ella Carol",4778.25,Ella,Abelli-Amen
3,"Abeloos, Diego V",67907.67,Diego,Abeloos
4,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby
...,...,...,...,...
4728,"Zook, Garet R",63830.44,Garet,Zook
4729,"Zumalt, Julia Mae",3311.86,Julia,Zumalt
4730,"Zumbro, Emma Catherine",6752.28,Emma,Zumbro
4731,"Zuzow, Lynnetta Robin",68914.67,Lynnetta,Zuzow


In [177]:
duplicates_rating = df_ratings.groupby(['firstName', 'lastName']).filter(lambda x: len(x) > 1)
print("There is " + str(len(duplicates_rating)) + " duplicates first and last names in the rating dataframe")

duplicates_payroll = df_payroll.groupby(['firstName', 'lastName']).filter(lambda x: len(x) > 1)
print("There is " + str(len(duplicates_payroll)) + " duplicates first and last names in the payroll dataframe")

There is 9 duplicates first and last names in the rating dataframe
There is 19 duplicates first and last names in the payroll dataframe


This is the duplicate rating dataframe, where through meticulous investigation, we found out that, Chris Clark is the same professor that teaches two department, Mary Armstrong are two different professor that doesn't teach anymore, Michael Miller are two different professor, where one teaches and one doesn't, and Todd Hagobian that are the same professor that teaches.

In [178]:
duplicates_rating.sort_values(by='firstName', inplace=True)
duplicates_rating

,lastName,firstName,department,numEvals,overallRating,materialClear,studentDifficulties,courses
1,Clark,Chris,CSC,12,3.70,3.62,3.70,"[CPE 103, CSC 103, CPE 123, CRP 430, CPE 481, ..."
345,Clark,Chris,CRP,16,3.75,3.75,3.75,"[CRP 101, CRP 212, CRP 404, CRP 408, CRP 412, ..."
957,Armstrong,Mary,LA,4,3.20,3.20,3.20,"[WS 301, BUS 322]"
1244,Armstrong,Mary,ENGL,21,3.59,3.73,3.64,"[WS 301, WS 310, ENGL 334, WS 340, ENGL 345, W..."
62,Miller,Michael,ART,33,2.79,2.97,2.59,"[ART 101, ART 201, ART 203, ART 301, ART 335, ..."
728,Miller,Michael,MATH,8,3.62,3.75,3.74,"[MATH 141, MATH 142]"
1601,Miller,Michael,MATH,1,4.00,4.00,4.00,[MATH]
1013,Hagobian,Todd,KINE,3,3.40,3.40,3.00,[KINE 303]
1577,Hagobian,Todd,KINE,5,3.17,3.17,3.33,[KINE 303]


Therefore we decided to drop the two Mary Armstrong and the Micheal Miller from ART department, besides from that, we also decided to combine the same professor to ensure data consistency from the original dataframe.

In [179]:
df_ratings.drop(index=62, inplace=True)
df_ratings.drop(index=957, inplace=True)
df_ratings.drop(index=1244, inplace=True)

In [180]:
# Compute product of evaluations and ratings for weighted average calculation
df_ratings['weightedOverallRating'] = df_ratings['numEvals'] * df_ratings['overallRating']
df_ratings['weightedMaterialClear'] = df_ratings['numEvals'] * df_ratings['materialClear']
df_ratings['weightedStudentDifficulties'] = df_ratings['numEvals'] * df_ratings['studentDifficulties']

# Aggregating function
aggregation_functions = {
    'department': 'first',  # Keep the first department
    'numEvals': 'sum',
    'weightedOverallRating': 'sum',
    'weightedMaterialClear': 'sum',
    'weightedStudentDifficulties': 'sum',
    'courses': lambda x: list(set(sum(x, [])))  # Flatten list and remove duplicates
}

# Group by first and last name and aggregate
df_ratings = df_ratings.groupby(['firstName', 'lastName']).agg(aggregation_functions)

# Calculate weighted averages
df_ratings['overallRating'] = df_ratings['weightedOverallRating'] / df_ratings['numEvals']
df_ratings['materialClear'] = df_ratings['weightedMaterialClear'] / df_ratings['numEvals']
df_ratings['studentDifficulties'] = df_ratings['weightedStudentDifficulties'] / df_ratings['numEvals']

# Drop weighted columns as they are no longer needed
df_ratings.drop(columns=['weightedOverallRating', 'weightedMaterialClear', 'weightedStudentDifficulties'], inplace=True)

# Reset index to make firstName and lastName columns again
df_ratings.reset_index(inplace=True)

df_ratings

,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties
0,Lubomir,Stanchev,CSC,2,"[CSC 365, DATA 301]",3.00,2.00,3.50
1,Aaron,Estrada,PSY,34,"[PSY 555, PSY 323, PSY 305, PSY 370, PSY 252, ...",2.58,2.73,2.58
2,Aaron,Keen,CSC,90,"[CPE 317, CPE 430, CSC 330, CSC 102, CPE 357, ...",2.88,2.89,2.77
3,Aaron,Newcomer,MATH,15,"[MATH 143, MATH 142]",3.00,3.15,2.85
4,Aaron,Rodriguez,ES,35,"[ES 210, ES 110, ES 114, ES 243, SOC 316, ES 3...",3.37,3.43,3.17
...,...,...,...,...,...,...,...,...
1851,jnan,blau,COMS,24,"[COMS 101, COMS 416, COMS 316, SPC 208, SPC 41...",3.60,3.66,3.34
1852,robert,schaeffer,SOC,18,"[SOC 110, SOC 218, SOC 118, SOC 395]",3.50,3.17,3.44
1853,sasha,poma,ENGL,2,"[ENGL 133, ENGL 134]",4.00,4.00,4.00
1854,virginia,callow,IME,36,"[IME 320, IME 443, IME 410, IME 418, IME 319, ...",2.12,1.97,2.15


As we can see now, there is no longer any duplicates in the rating dataframe.

In [181]:
duplicates_rating = df_ratings.groupby(['firstName', 'lastName']).filter(lambda x: len(x) > 1)
print("There is " + str(len(duplicates_rating)) + " duplicates first and last names in the rating dataframe")

There is 0 duplicates first and last names in the rating dataframe


After clearing out duplicates and ensuring the data consistency within the rating dataframe, we should do the same with the payroll dataframe.

In [182]:
duplicates_payroll.sort_values(by='firstName', inplace=True)
duplicates_payroll

,name,avgPay,firstName,lastName
3652,"Russell, Craig H",47821.00,Craig,Russell
3653,"Russell, Craig Nolan",67439.39,Craig,Russell
2668,"Marshall, David C",64006.75,David,Marshall
2669,"Marshall, David D",152803.00,David,Marshall
2670,"Marshall, David James",115574.67,David,Marshall
3712,"Sanchez, David Cardenas",499.90,David,Sanchez
3711,"Sanchez, David A",12752.00,David,Sanchez
2032,"Johnson, Eric James",24298.45,Eric,Johnson
2033,"Johnson, Eric R",97206.38,Eric,Johnson
100,"Anderson, Jeffery Lynn",32264.61,Jeffery,Anderson


First we can see if all the people in the duplicates have ratings, therefore we can do a left merge, if the person doesn't have ratings, we can just drop them, since they might not be professor, but just a faculty member.

In [183]:
duplicates_payrollRating = duplicates_payroll.merge(df_ratings, on=['firstName', 'lastName'], how="left")
na_payrollRating = duplicates_payrollRating[duplicates_payrollRating['overallRating'].isna()]
na_payrollRating

,name,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties
5,"Sanchez, David Cardenas",499.90,David,Sanchez,NaN,NaN,NaN,NaN,NaN,NaN
6,"Sanchez, David A",12752.00,David,Sanchez,NaN,NaN,NaN,NaN,NaN,NaN
7,"Johnson, Eric James",24298.45,Eric,Johnson,NaN,NaN,NaN,NaN,NaN,NaN
8,"Johnson, Eric R",97206.38,Eric,Johnson,NaN,NaN,NaN,NaN,NaN,NaN
9,"Anderson, Jeffery Lynn",32264.61,Jeffery,Anderson,NaN,NaN,NaN,NaN,NaN,NaN
10,"Anderson, Jeffery M",76299.08,Jeffery,Anderson,NaN,NaN,NaN,NaN,NaN,NaN
13,"Grimes, Joseph E",56073.17,Joseph,Grimes,NaN,NaN,NaN,NaN,NaN,NaN
14,"Grimes, Joseph Edward",13166.68,Joseph,Grimes,NaN,NaN,NaN,NaN,NaN,NaN


After we know that these are not professors, therefore we can just drop them thorugh using their index.

In [184]:
df_payroll.drop(index=3711, inplace=True)
df_payroll.drop(index=3712, inplace=True)
df_payroll.drop(index=2032, inplace=True)
df_payroll.drop(index=2033, inplace=True)
df_payroll.drop(index=100, inplace=True)
df_payroll.drop(index=101, inplace=True)
df_payroll.drop(index=1590, inplace=True)
df_payroll.drop(index=1591, inplace=True)

Now we want to focus on the names that actually have ratings

In [185]:
notna_payrollRating = duplicates_payrollRating[duplicates_payrollRating['overallRating'].notna()]

In [186]:
notna_payrollRating

,name,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties
0,"Russell, Craig H",47821.00,Craig,Russell,MU,107.0,"[MU 120, MU 101, MU 324, MU 121, MU 229, MU 32...",3.660000,3.560000,3.580000
1,"Russell, Craig Nolan",67439.39,Craig,Russell,MU,107.0,"[MU 120, MU 101, MU 324, MU 121, MU 229, MU 32...",3.660000,3.560000,3.580000
2,"Marshall, David C",64006.75,David,Marshall,AERO,17.0,"[AERO 525, AERO 306, AERO 301, AERO 304, AERO ...",2.060000,2.560000,1.940000
3,"Marshall, David D",152803.00,David,Marshall,AERO,17.0,"[AERO 525, AERO 306, AERO 301, AERO 304, AERO ...",2.060000,2.560000,1.940000
4,"Marshall, David James",115574.67,David,Marshall,AERO,17.0,"[AERO 525, AERO 306, AERO 301, AERO 304, AERO ...",2.060000,2.560000,1.940000
11,"Oliver, John Edman",5237.67,John,Oliver,CPE,2.0,"[CPE 350, CPE 100]",4.000000,4.000000,4.000000
12,"Oliver, John Y",106037.33,John,Oliver,CPE,2.0,"[CPE 350, CPE 100]",4.000000,4.000000,4.000000
15,"Williams, Kevin Todd",68301.42,Kevin,Williams,IME,35.0,"[IME 141, IME 142]",3.730000,3.180000,3.560000
16,"Williams, Kevin Thomas",83864.03,Kevin,Williams,IME,35.0,"[IME 141, IME 142]",3.730000,3.180000,3.560000
17,"Miller, Michael G",53198.59,Michael,Miller,MATH,9.0,"[MATH, MATH 142, MATH 141]",3.662222,3.777778,3.768889


Through meticulous investigation by looking at the professor's latest rating from polyratings, I was able to find the professor's full name through the course they used to teach through this website https://schedules.calpoly.edu/index_2224.htm

real professors within this dataframe is Marshall, David D, Russell, Craig H, Oliver, John Y, Williams, Kevin Todd, Miller, Michael G

therefore we can just drop other ones that are not a professor

In [187]:
df_payroll.drop(index=3653, inplace=True)
df_payroll.drop(index=2668, inplace=True)
df_payroll.drop(index=2670, inplace=True)
df_payroll.drop(index=3121, inplace=True)
df_payroll.drop(index=4574, inplace=True)
df_payroll.drop(index=2893, inplace=True)

In [188]:
df_payroll

,name,avgPay,firstName,lastName
0,"Aagaard, Sofie J",91650.96,Sofie,Aagaard
1,"Abba, Michele M",57789.41,Michele,Abba
2,"Abelli-Amen, Ella Carol",4778.25,Ella,Abelli-Amen
3,"Abeloos, Diego V",67907.67,Diego,Abeloos
4,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby
...,...,...,...,...
4728,"Zook, Garet R",63830.44,Garet,Zook
4729,"Zumalt, Julia Mae",3311.86,Julia,Zumalt
4730,"Zumbro, Emma Catherine",6752.28,Emma,Zumbro
4731,"Zuzow, Lynnetta Robin",68914.67,Lynnetta,Zuzow


In [189]:
duplicates_payroll = df_payroll.groupby(['firstName', 'lastName']).filter(lambda x: len(x) > 1)
print("There is " + str(len(duplicates_payroll)) + " duplicates first and last names in the payroll dataframe")

There is 0 duplicates first and last names in the payroll dataframe


Through these, we have successfully clear out the dataframe and ensured data consistency.

In [190]:
print("The number of payroll is " + str(len(df_payroll)))
print("The number of rating is " + str(len(df_ratings)))
df_payRating = df_payroll.merge(df_ratings, on=["firstName", "lastName"])

The number of payroll is 4719
The number of rating is 1856


In [192]:
df_payRating

,name,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,"[AERO 353, AERO 452, AERO 351, AERO 354, AERO ...",3.39,3.34,3.42
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,"[PHIL 201, PHIL 336, PHIL 231, PHIL 125, PHIL ...",2.31,2.61,2.05
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,"[BIO 405, BIO 440, MSC 440, ZOO 336, BIO 160, ...",2.82,2.68,2.75
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,"[ART 112, ART 312, ART 315, ART 311, WGS 210, ...",3.56,3.66,3.48
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,"[EE 212, EE 151, EE 306, EE 418, EE 314, EE 30...",1.23,1.28,1.01
...,...,...,...,...,...,...,...,...,...,...
890,"Zhang, Shanju",117869.34,Shanju,Zhang,CHEM,25,"[CHEM 125, CHEM 128, CHEM 351, CHEM 129, CHEM ...",2.88,2.81,2.77
891,"Zigler, David F",95367.33,David,Zigler,CHEM,13,"[CHEM 125, CHEM 127, CHEM 126, CHEM 124, CHEM ...",1.16,1.23,1.38
892,"Zoerb, Matthew C",98424.00,Matthew,Zoerb,CHEM,1,[CHEM],3.00,3.00,3.00
893,"Zoller, David J",74328.67,David,Zoller,PHIL,13,"[PHIL 331, PHIL 335, PHIL 230, PHIL 231]",3.23,3.08,3.08


next step, get the professor list, match it on, give the na lecture/ta

Information scraped off from https://catalog.calpoly.edu/facultyandstaff/#facultystaffemeritustext

In [193]:
df_position = pd.read_csv('position.csv')

In [194]:
df_position['name'] = df_position['name'].str.replace('.','')

In [195]:
for index, row in df_position.iterrows():
    name = row['name']
    lenName = len(name.split(", "))
    if lenName == 2:
      lastname, firstname_middle = name.split(", ")
    elif lenName == 3:
      lastname, firstname_middle, _ = name.split(", ")
      print(name)
    elif lenName == 4:
      lastname, firstname_middle, _, _ = name.split(", ")
      print(name)
    firstname_parts = firstname_middle.split()
    firstname = firstname_parts[0]
    df_position.loc[index, 'firstName'] = firstname
    df_position.loc[index, 'lastName'] = lastname

Castilow, Dan C, II
Fowler, Thomas , IV, FAIA


In [196]:
df_position

,name,year,dept,position,education,firstName,lastName
0,"Abercromby, Kira J",2008,Aerospace Engineering,Professor,"B.S., University of California, Los Angeles, 1...",Kira,Abercromby
1,"Abo Ismail, Mohammed",2019,Animal Science,Assistant Professor,"B.S., Alexandria University Damanhour, 2000; M...",Mohammed,Abo Ismail
2,"Adams, Nikki L",2002,Biological Sciences,Professor,"B.A., University of California, Santa Barbara,...",Nikki,Adams
3,"Adams, Aubrie S",2017,Communication Studies,Associate Professor,"B.A., Sonoma State University, 2005; M.A., Sac...",Aubrie,Adams
4,"Adams, Elizabeth",2023,Construction Management,Assistant Professor,"B.S., Boise State University, 2001; M.S., Univ...",Elizabeth,Adams
...,...,...,...,...,...,...,...
773,"Shani, Abraham (Rami) B",1983,"Management, HR, and Information Systems",Professor Emeritus (FERP),"B.A., University of Tel Aviv, 1972; M.A., 1978...",Abraham,Shani
774,"Singh, Jagjit J",2003,Industrial Technology and Packaging,Professor,"B.S., Poona University, Pune, India, 1992; M.S...",Jagjit,Singh
775,"Yin, Bingqing M",2021,Marketing,Assistant Professor,"Ph.D., University of Kansas 2019, M.S. Busines...",Bingqing,Yin
776,"Yu, Xiao-Hua H",2000,Electrical Engineering,Professor,"B.S., TianJin University, People’s Republic of...",Xiao-Hua,Yu


In [197]:
duplicates_position = df_position.groupby(['firstName', 'lastName']).filter(lambda x: len(x) > 1)
print("There is " + str(len(duplicates_position)) + " duplicates first and last names in the position dataframe")

There is 0 duplicates first and last names in the position dataframe


In [198]:
df_payRating

,name,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,"[AERO 353, AERO 452, AERO 351, AERO 354, AERO ...",3.39,3.34,3.42
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,"[PHIL 201, PHIL 336, PHIL 231, PHIL 125, PHIL ...",2.31,2.61,2.05
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,"[BIO 405, BIO 440, MSC 440, ZOO 336, BIO 160, ...",2.82,2.68,2.75
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,"[ART 112, ART 312, ART 315, ART 311, WGS 210, ...",3.56,3.66,3.48
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,"[EE 212, EE 151, EE 306, EE 418, EE 314, EE 30...",1.23,1.28,1.01
...,...,...,...,...,...,...,...,...,...,...
890,"Zhang, Shanju",117869.34,Shanju,Zhang,CHEM,25,"[CHEM 125, CHEM 128, CHEM 351, CHEM 129, CHEM ...",2.88,2.81,2.77
891,"Zigler, David F",95367.33,David,Zigler,CHEM,13,"[CHEM 125, CHEM 127, CHEM 126, CHEM 124, CHEM ...",1.16,1.23,1.38
892,"Zoerb, Matthew C",98424.00,Matthew,Zoerb,CHEM,1,[CHEM],3.00,3.00,3.00
893,"Zoller, David J",74328.67,David,Zoller,PHIL,13,"[PHIL 331, PHIL 335, PHIL 230, PHIL 231]",3.23,3.08,3.08


In [199]:
df_payPositionRating = df_payRating.merge(df_position, on=['firstName', 'lastName'], how='left')
df_payPositionRating['position'].fillna('Lecturer/TA', inplace=True)

In [200]:
df_payPositionRating

,name_x,avgPay,firstName,lastName,department,numEvals,courses,overallRating,materialClear,studentDifficulties,name_y,year,dept,position,education
0,"Abercromby, Kira Jorgensen",123885.33,Kira,Abercromby,AERO,20,"[AERO 353, AERO 452, AERO 351, AERO 354, AERO ...",3.39,3.34,3.42,"Abercromby, Kira J",2008,Aerospace Engineering,Professor,"B.S., University of California, Los Angeles, 1..."
1,"Abney, Keith A",99122.78,Keith,Abney,PHIL,138,"[PHIL 201, PHIL 336, PHIL 231, PHIL 125, PHIL ...",2.31,2.61,2.05,NaN,NaN,NaN,Lecturer/TA,NaN
2,"Adams, Nikki L",116394.67,Nikki,Adams,BIO,27,"[BIO 405, BIO 440, MSC 440, ZOO 336, BIO 160, ...",2.82,2.68,2.75,"Adams, Nikki L",2002,Biological Sciences,Professor,"B.A., University of California, Santa Barbara,..."
3,"Adan, Elizabeth",130039.99,Elizabeth,Adan,ART,37,"[ART 112, ART 312, ART 315, ART 311, WGS 210, ...",3.56,3.66,3.48,"Adan, Elizabeth",2007,"Women's, Gender and Queer Studies, Interdiscip...",Professor and WGQS Department Chair,"B.A., University of California, Davis, 1993; M..."
4,"Agbo, Samuel O",85824.17,Samuel,Agbo,EE,80,"[EE 212, EE 151, EE 306, EE 418, EE 314, EE 30...",1.23,1.28,1.01,"Agbo, Samuel O",1991,Electrical Engineering,Professor,"B.Sc., University of Nigeria, 1975; M.S.E., Un..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,"Zhang, Shanju",117869.34,Shanju,Zhang,CHEM,25,"[CHEM 125, CHEM 128, CHEM 351, CHEM 129, CHEM ...",2.88,2.81,2.77,"Zhang, Shanju",2011,Chemistry and Biochemistry,Professor,"B.S., Jilin University, People's Republic of C..."
891,"Zigler, David F",95367.33,David,Zigler,CHEM,13,"[CHEM 125, CHEM 127, CHEM 126, CHEM 124, CHEM ...",1.16,1.23,1.38,"Zigler, David F",2016,Chemistry and Biochemistry,Associate Professor,"B.S., Eastern Illinois University; Ph.D., Virg..."
892,"Zoerb, Matthew C",98424.00,Matthew,Zoerb,CHEM,1,[CHEM],3.00,3.00,3.00,"Zoerb, Matthew C",2015,Chemistry and Biochemistry,Associate Professor,"B.S., UT Chattanooga, 2006; Ph.D., Univeristy ..."
893,"Zoller, David J",74328.67,David,Zoller,PHIL,13,"[PHIL 331, PHIL 335, PHIL 230, PHIL 231]",3.23,3.08,3.08,"Zoller, David J",2015,Philosophy,Associate Professor,"B.A. Xavier University, 2003; M.A., Fordham Un..."
